In [4]:
import csv
import numpy as np
import pandas as pd
import utils 
import seaborn as sns

from scipy.optimize import minimize, LinearConstraint
from scipy.special import softmax

In [5]:
data = np.genfromtxt('data_new.csv', delimiter=',', dtype=str)
data = np.delete(data, 0, 0)

In [6]:
data.shape

(48, 17)

In [7]:
new_data = []

# turn each game into entry in this array

def remove_blank(l):
    return list(filter(lambda a: a != '', l))

def replace_letters(l):
   return [[1,0] if x=='c' else [0, 1] for x in l]

for i in range(0, data.shape[0], 2):
    p1 = data[i]
    p2 = data[i+1]

    game = int(p1[0])
    player1 = p1[2]
    player2 = p2[2]
    p1_actions = np.array(replace_letters(remove_blank(p1[3:]))).T
    p2_actions =  np.array(replace_letters(remove_blank(p2[3:]))).T
   
    actions =  [p1_actions, p2_actions]

    new_data.append([game, player1, player2, actions])


In [8]:
df = pd.DataFrame(new_data)
df.columns = ['Game', 'Player_1', 'Player_2', 'Actions']

In [9]:
df.head()

,Game,Player_1,Player_2,Actions
0,0,Faisal,Bharathvaj,"[[[0, 1, 1, 1, 0], [1, 0, 0, 0, 1]], [[1, 0, 1..."
1,1,yuan,Dagmar,"[[[1, 0, 0], [0, 1, 1]], [[1, 1, 0], [0, 0, 1]]]"
2,2,Ehsan,Mohammend,"[[[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0], [0, 0,..."
3,3,Tao,Rohini,"[[[0, 1, 0, 1, 1], [1, 0, 1, 0, 0]], [[1, 0, 0..."
4,4,Anita,Greg,"[[[1, 1, 0, 1, 1, 1, 0], [0, 0, 1, 0, 0, 0, 1]..."


In [10]:
s1 = np.zeros(2)
s2 = np.zeros(2)


for i, row in df.iterrows():
    actions = row.Actions
    
    for round in range( actions[0].shape[1]): # iterate over length of each game 
        s1 += actions[0][:, round]
        s2 += actions[1][:, round]
        
    # frequency of each
print(s1/np.sum(s1))
print(s2/np.sum(s2))

[0.40151515 0.59848485]
[0.48484848 0.51515152]


In [11]:
def p_traj(a, s):
    '''
    Returns the probability of a sequence of actions
    a being sampled from strategy s.

    Paramters:
        a : (np.Array) 2 x n array of actions chosen
        s : (np.Array) 2 x n array of strategies 
    '''

    assert a.shape == s.shape
    L = a.shape[1]
    prod = 1
    for l in range(L):
        idx = np.where(a[:, l])
        prod *= s[:, l][idx][0]

    return prod

In [12]:
data = list(df.Actions)
traj = data[0][0][:, 0:3]
s = np.array([[0.9, 0.1], [0.6, 0.4], [.3, .7] ]).T
assert p_traj(traj, s) == 0.018

In [13]:
def p_outcomes(s):
    '''
    returns a probability distribution over outcomes
    '''
    p = s[0]
    for s_i in s[1:]:
        p =  np.multiply.outer(p, s_i)
    return p

def expected_utility(s, payoffs):
    '''
    Gets the expected utilitiy for a player under strategy profile s
    
    Parameters
        s : (list) list where S[i] gives the probability of player i playing each action
        payoffs: (np.array) payoffs for i
    Returns
        eu : (float) expected utility for i
    '''
    p = p_outcomes(s)
    return np.sum(np.multiply(p, payoffs))


In [14]:
def ll_uniform( dataset):
    ll = 0
    for play in dataset: 
        for player in range(2):
            traj = play[player]
            s = np.ones_like(traj) / np.ones(traj.shape[1])*2
            ll += np.log(p_traj(traj, s))
    return -ll

ll_uniform(data)

-182.99085566782566

In [15]:
class Spite_Model:
    def __init__(self, h):
        self.h = h
        self.payoffs = np.zeros([2, 2, 2])
        self.payoffs[0, :, :] = np.array([[0, -1],
                                          [1, -2]])
        
        self.payoffs[1, :, :] = np.array([[0, 1],
                                         [-1, -2]])
        self.alphas = None
    

    def log_likelihood_independent(self, params, dataset):
        '''
        Given the dataset and the fitted model, returns the log likelihood. This assumes a players level may change as the game goes on

        NOTE: assumes each player has a fixed level
        
        Parameters:
            params : np.array of alpha_ks, the freq of that level in population and lambda_ for QBR
            dataset : list of plays, each play is an np array
        Returns:
            ll : loglikelihood of dataset

        '''
        
        alphas = params[0:-2]
        lambda_ = params[-2] 
        kappa  = params[-1] 

        ll = 0
        for play in dataset: 
            for player in range(2):
                traj = play[player]
                other_traj  = play[1-player] # trajectory of other player
                
                pred_s = self.predict_traj(traj, other_traj, self.K, lambda_, kappa, overall=True, alphas=alphas)  
               
                L = traj.shape[1] #length of trajectory
                for l in range(L):
                    idx = np.where(traj[:, l])
                    ll += np.log(pred_s[:, l][idx][0])

        return -ll # since we are minimizing the negative log likelihood


    def log_likelihood(self, params, dataset):
        '''
        Given the dataset and the fitted model, returns the log likelihood.

        NOTE: assumes each player has a fixed level
        
        Parameters:
            params : np.array of alpha_ks, the freq of that level in population and lambda_ for QBR
            dataset : list of plays, each play is an np array
        Returns:
            ll : loglikelihood of dataset

        '''
        spite_coeff = params[0] # weighting between spiteful strategy and QBR
        lambda_ = params[1]
       
        ll = 0
        for play in dataset: 
            for player in range(2):
                traj = play[player]
                other_traj  = play[1-player] # trajectory of other player
                
                pred_s = self.predict_traj(traj, other_traj, spite_coeff, lambda_)  
               
                L = traj.shape[1] #length of trajectory
                for l in range(L):
                    idx = np.where(traj[:, l])
                    ll += np.log(pred_s[:, l][idx][0]+0.00001)


        return -ll # since we are minimizing the negative log likelihood


    def fit(self, dataset):
        num_params = 2

        params = np.zeros(num_params) # +2 since level 0 and the lambda parameter, kappa parameter



        #const_arr = np.ones(num_params)
        #constraint = LinearConstraint(const_arr, lb=1, ub=1)
        bnds = [(0, 1) , (0, 10)]
        #bnds = [(0, 10) for x in range(1)]
    

        result = minimize(
            self.log_likelihood ,
            params, 
            args=(dataset),
            bounds=bnds) 

        assert result.status == 0 # make sure the optimization was successful
        return result


    def predict_traj(self, traj, other_traj, spite_coeff, lambda_):
        '''
        Returns straetgy predictions against other player
        
        '''

        play_len = traj.shape[1]
        pred_i = np.zeros((2, play_len))

        for l in range(play_len):
            if l ==0:
                pred_i[:, l] = np.ones(2)/2
               
            else:
                if np.array_equal(other_traj[:, l-1], np.array([0, 1])):
                    pred_i[:, l] = (1-spite_coeff) * self.compute_BR(other_traj[:, l-1], lambda_) + spite_coeff * np.array([0,1])
                else:
                    # print(pred_i.shape)
                    # print(other_traj.shape)
                    pred_i[:, l] = self.compute_BR(other_traj[:, l-1], lambda_)

        return pred_i   


    def compute_BR(self,  s_other, lambda_):
        '''
        Computes a best response

        Parameters:
            s_other : (np.Array) strategy of other player

        NOTE: this ONLY works with symetric payoffs and 2 actions
        '''
    
        #get EU of action 0
        s = [np.array([1, 0]), s_other]
        eu_0 = expected_utility(s, self.payoffs[0])

        # get EU of action 1
        s = [np.array([0, 1]), s_other]
        eu_1 = expected_utility(s, self.payoffs[0])

        # return action with greater EU
        return softmax(np.array([eu_0, eu_1])*lambda_)



In [288]:
class Bounded_Memory_Map:
    def __init__(self, h):
        self.h = h
        self.payoffs = np.zeros([2, 2, 2])
        self.payoffs[0, :, :] = np.array([[0, -1],
                                 [1, -2]])
        
        self.payoffs[1, :, :] = np.array([[0, 1],
                                        [-1, -2]])
        self.alphas = None

    def log_likelihood(self, params, dataset):
        '''
        Given the dataset and the fitted model, returns the log likelihood.

        NOTE: assumes each player has a fixed level
        
        Parameters:
            params : np.array of alpha_ks, the freq of that level in population and lambda_ for QBR
            dataset : list of plays, each play is an np array
        Returns:
            ll : loglikelihood of dataset

        '''       
        num_features = len(self.get_features(0, 1))
        M = params[:-1].reshape((num_features, num_features))
        lambda_ = params[-1]


        ll = 0
        for play in dataset: 
            for player in range(2):
                traj = play[player]
                other_traj  = play[1-player] # trajectory of other player
                
                pred_s = self.predict_traj(traj, other_traj, M, player,  lambda_)  
               
                L = traj.shape[1] #length of trajectory
                for l in range(L):
                    idx = np.where(traj[:, l])
                    ll += np.log(pred_s[:, l][idx][0]+0.00001)


        return -ll # since we are minimizing the negative log likelihood


    def fit(self, dataset):
        num_features = len(self.get_features(0, 1))

        # additional weights for subhistories

        params = np.random.random(num_features*num_features+1)
        bnds = [(0, None) for i in range(params.shape[0]-1)]
        bnds.append((0, 10))
    

        result = minimize(
            self.log_likelihood ,
            params, 
            bounds = bnds,
            args=(dataset)) 
      
        print(result)
        assert result.status == 0 # make sure the optimization was successful
        return result



    def get_features(self, player, lambda_):
        '''
        returns a set of linear_4 like features
        '''

        def strat(a):
            nonlocal lambda_
            weight = 1/len(a)
            s = np.zeros(2)
            for a_ in a:
                s[a_] = weight
            return softmax(s*lambda_)

        payoffs = self.payoffs

        other_player = 1-player


        # Problem: if, between switches, 


        def all_argmax(a):
            return np.argwhere(a == np.amax(a)).flatten().tolist()

        def all_argmin(a):
            return np.argwhere(a == np.amin(a)).flatten().tolist()


        if player == 0:
            max_min = strat(all_argmax(np.min(payoffs[player], axis=1)))
            max_max = strat(all_argmax(np.max(payoffs[player], axis=1)))
            min_max = strat(all_argmin(np.max(payoffs[other_player], axis=1)))
            
            eff = strat(all_argmax(np.max(payoffs[player] + payoffs[other_player], axis=1)))
            fair = strat(all_argmin(np.min( np.abs(payoffs[player] - payoffs[other_player]), axis=1)))

        else:
            max_min = strat(all_argmax(np.min(payoffs[player], axis=0)))
            max_max = strat(all_argmax(np.max(payoffs[player], axis=0)))
            min_max = strat(all_argmin(np.max(payoffs[other_player], axis=0)))
            
            eff = strat(all_argmax(np.max(payoffs[player] + payoffs[other_player], axis=0)))
            fair = strat(all_argmin(np.min( np.abs(payoffs[player] - payoffs[other_player]), axis=0)))

        unif = np.ones(2)/2

        return [max_min, max_max, min_max, eff, fair, unif]


    def hist_to_str(hist):
        hist = hist.astype(str).tolist()
        return [''.join(elmt) for elmt in hist]


    def predict_traj(self, traj, other_traj, M, player,  lambda_, print_=False):
        '''
        Returns straetgy predictions against other player
        
        '''
        play_len = traj.shape[1]
        pred_i = np.zeros((2, play_len))

        for l in range(play_len):
            if l == 0:
                pred_i[:, l] = np.ones(2)/2
               
            else:
                p = self.get_probs(other_traj[:, max(l-self.h, 0):l], 1-player, lambda_) # probability of history under each simple strat
               
                p = p / np.max(p) # p is the same
                w = np.dot(M, p) # w is the same 
              
                features = self.get_features(player, lambda_)  

                if print_:
                    print(w)
                    print(p)
                    print(features)
                weighted_features = np.zeros((2, len(features)))


                sum_w = np.sum(w)
                for i in range(len(features)):
                    weighted_features[:, i] = features[i] * (w[i]/sum_w)
              
                pred_i[:, l] = np.sum(np.array(weighted_features), axis = 1)
            
        return pred_i   



    def get_probs(self, hist, player, lambda_):
        '''
        Returns the probability of a history 
        '''


        def p_traj_single(a, s):
            '''
            Returns the probability of a sequence of actions
            a being sampled from strategy s.

            Paramters:
                a : (np.Array) 2 x n array of actions chosen
                s : (np.Array) 2 x 1 strategy
            '''
            L = a.shape[1]
            prod = 1
            for l in range(L):
                idx = np.where(a[:, l])
                prod *= s[idx][0]

            return prod

        features = self.get_features(player, lambda_)
        probs = np.zeros(len(features))
        avg_h = np.mean(hist, axis=1 )
       
        for i, s in enumerate(features):
            probs[i] = p_traj_single(hist, s)

        return probs


    def compute_BR(self,  s_other, lambda_):
        '''
        Computes a best response

        Parameters:
            s_other : (np.Array) strategy of other player

        NOTE: this ONLY works with symetric payoffs and 2 actions
        '''
    
        #get EU of action 0
        s = [np.array([1, 0]), s_other]
        eu_0 = expected_utility(s, self.payoffs[0])

        # get EU of action 1
        s = [np.array([0, 1]), s_other]
        eu_1 = expected_utility(s, self.payoffs[0])

        # return action with greater EU
        return softmax(np.array([eu_0, eu_1])*lambda_)

In [290]:
m = Bounded_Memory_Map(2)
r = m.fit(data)

In [253]:
m2 = deepcopy(m)
m2.payoffs[0][:, [0, 1]] = m.payoffs[0][:, [1, 0]]
m2.payoffs[1][:, [0, 1]] = m.payoffs[1][:, [1, 0]]

array([[[-1.,  0.],
        [-2.,  1.]],

       [[ 1.,  0.],
        [-2., -1.]]])

In [254]:
m.get_probs(new_data[1][0], 1)

array([0.   , 0.   , 0.   , 0.125, 0.125])

In [255]:
m.get_probs(data[1][0], 1)

array([0.   , 0.   , 0.   , 0.125, 0.125])

In [262]:
# flip actions for player 1

from copy import deepcopy
new_data = deepcopy(data)
for play in new_data:
    # for p in range(2):
    #play[0][[0, 1]] = play[0][[1, 0]]
    play[1][[0, 1]] = play[1][ [1, 0]]
   # play[1][ [0, 1], :] = play[1][ [1, 0], :]


In [263]:
data[0]

[array([[0, 1, 1, 1, 0],
        [1, 0, 0, 0, 1]]),
 array([[1, 0, 1, 0, 0],
        [0, 1, 0, 1, 1]])]

In [264]:
new_data[0]

[array([[0, 1, 1, 1, 0],
        [1, 0, 0, 0, 1]]),
 array([[0, 1, 0, 1, 1],
        [1, 0, 1, 0, 0]])]

In [265]:
m.log_likelihood(r.x, data)

160.88443463824265

In [258]:
m2.payoffs

array([[[-1.,  0.],
        [-2.,  1.]],

       [[ 1.,  0.],
        [-2., -1.]]])

In [266]:
m2.log_likelihood(r.x, new_data)

160.88443463824265

In [225]:
m.predict_traj(data[0][0], data[0][1], r.x.reshape((3,3)), 0, print_=True)

[1.66095402 0.88219117 0.45197306]
[1.  0.  0.5]
[array([1., 0.]), array([0., 1.]), array([0.5, 0.5])]
[0.21946011 1.67593238 0.74440842]
[0.  1.  0.5]
[array([1., 0.]), array([0., 1.]), array([0.5, 0.5])]
[1.66095402 0.88219117 0.45197306]
[1.  0.  0.5]
[array([1., 0.]), array([0., 1.]), array([0.5, 0.5])]
[0.21946011 1.67593238 0.74440842]
[0.  1.  0.5]
[array([1., 0.]), array([0., 1.]), array([0.5, 0.5])]


array([[0.5       , 0.63000536, 0.22413217, 0.63000536, 0.22413217],
       [0.5       , 0.36999464, 0.77586783, 0.36999464, 0.77586783]])

In [226]:
m2.predict_traj(new_data[0][0], new_data[0][1], r.x.reshape((3,3)), 0, print_=True)

[1.66095402 0.88219117 0.45197306]
[1.  0.  0.5]
[array([0., 1.]), array([1., 0.]), array([0.5, 0.5])]
[0.21946011 1.67593238 0.74440842]
[0.  1.  0.5]
[array([0., 1.]), array([1., 0.]), array([0.5, 0.5])]
[1.66095402 0.88219117 0.45197306]
[1.  0.  0.5]
[array([0., 1.]), array([1., 0.]), array([0.5, 0.5])]
[0.21946011 1.67593238 0.74440842]
[0.  1.  0.5]
[array([0., 1.]), array([1., 0.]), array([0.5, 0.5])]


array([[0.5       , 0.36999464, 0.77586783, 0.36999464, 0.77586783],
       [0.5       , 0.63000536, 0.22413217, 0.63000536, 0.22413217]])

In [133]:
m.payoffs

array([[[ 1., -2.],
        [ 0., -1.]],

       [[-1., -2.],
        [ 0.,  1.]]])